# GAIA PSF Simulation

First attempt to simulate close binary stars observation using the gaia telescope. The latter is imagined as a rectangular pupil, with no pixel binning or aberrations

#### Initialization

In [1]:
# %matplotlib qt

In [8]:
import logging
logging.basicConfig(level=logging.INFO, format=' - %(levelname)s - %(message)s')

In [9]:
import poppy
import xupy as xp
from xupy import typing as xt
import astropy.units as u
from astropy import convolution as c
from astropy.io import fits
from astropy.table import QTable
from matplotlib import pyplot as plt
from opticalib import load_fits

Gpb = QTable.read("data/gaiaDR3passband.fits")
weights = Gpb.filled(0)

def rebinned(psf: fits.HDUList | xt.ArrayLike, rebin_factor: int) -> fits.HDUList:
    """
    Rebin PSF by a given factor, following Gaia's pixel scale (1:3 ratio).
    """
    if isinstance(psf, fits.HDUList):
        psf = psf[0].data
    px_ratio = (rebin_factor, 3*rebin_factor)
    return poppy.utils.rebin_array(psf, px_ratio)

```py
poppy.conf.use_multiprocessing = False
poppy.conf.n_processes = 20
poppy.conf.double_precision = False

M1 = poppy.RectangleAperture(name="Primary Mirror", width=1.45*u.m, height=0.5*u.m)
M2 = poppy.RectangleAperture(name="Beam Collimator / Secondary Mirror",width=0.35*u.m, height=0.2*u.m)
telescope = poppy.OpticalSystem(name="Gaia", oversample=1, verbose=True)

telescope.add_pupil(M1)
telescope.add_detector(name="SkyPLANE", pixelscale=0.001*(u.arcsec/u.pixel), fov_arcsec=1*u.arcsec, oversample=10)

psf = telescope.calc_psf(progressbar=True, return_final=True, source={'wavelengths': Gpb['lambda'], 'weights': weights['G']})

## Got The PSF: Now on to the convolution

In [10]:
from simulator import BinarySystem, CCD

#### Creating the Binary star system

In [11]:
bs = BinarySystem(M1=5, M2=7, distance=200)

bs._band_flux()
bs.create_raw_binary_cube(collecting_area=1.45*0.5*u.m**2,t_integration=4.42*u.s, shape=(440,440))


#### Loading the PSF and creating the CCD

In [12]:
psf_file = "data/simulations/PSFs/gaia_psf_2.fits"
psf, psf_H = load_fits(psf_file, return_header=True)

for key in psf_H.keys():
    if key == 'HISTORY':
        print(f"{key}: {psf_H[key]}")
        break
    if 'WAVE' not in key and 'WGHT' not in key:
        print(f"{key}: {psf_H[key]}")

ccd = CCD(psf_file)

SIMPLE: True
BITPIX: -32
NAXIS: 2
NAXIS1: 10000
NAXIS2: 10000
EXTEND: True
PLANE1: Wavefront Intensity
DIFFLMT: 0.04303385928527069
OVERSAMP: 1
DET_SAMP: 1
PIXELSCL: 0.0001
FOV: 1.0
FFTTYPE: numpy.fft
NORMALIZ: first
HISTORY: Created wavefront: wavelength=320 nm, diam=1.534 m
 using array size (1024, 1024)
  Multiplied WF by phasor for Pupil plane: Primary Mirror
  Propagating wavefront to Detector plane: SkyPLANE (1000x1000 pixels, 0
.001 arcsec / pix).
    Propagating w/ MFT: 0.0001 arcsec / pix     fov=23.238 lam/D    npix
=10000
Calculation completed in 127.198 seconds


**First try at observation**

In [ ]:
res = bs.observe(ccd)